In [7]:
from utils import Lung_Dataset, AugmentedDataset

In [8]:
train_set = Lung_Dataset("train", base_dir="")
test_set = Lung_Dataset("test")
val_set = Lung_Dataset("val")

FileNotFoundError: [WinError 3] The system cannot find the path specified: './dataset/train/normal/'

In [9]:
train_set.show_img("normal", 1)

NameError: name 'train_set' is not defined

In [10]:
train_set.show_img("covid", 1)

NameError: name 'train_set' is not defined

In [11]:
train_set.show_img("non-covid", 1)

NameError: name 'train_set' is not defined

In [12]:
train_set = Lung_Dataset("train", transform=1)
train_set1 = Lung_Dataset("train", transform=2)
train_set2 = Lung_Dataset("train", transform=4)
augmented_set = AugmentedDataset(train_set, train_set2, train_set1)

FileNotFoundError: [WinError 3] The system cannot find the path specified: './dataset/train/normal/'

In [13]:
augmented_set.show_img("train", 4, 1, 1, 1)

NameError: name 'augmented_set' is not defined